In [1]:
import sqlite3
conn = sqlite3.connect('data/ch10_data.db')
cursor = conn.cursor()

In [14]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def as_dict(self): #insert할 때 사용
        return {"name":self.name, "phone":self.phone, 
                "email":self.email, "age":self.age, "grade":self.grade, 
                "etc":self.etc}
    def __str__(self):
        return "{:>5} {:3} {:15} {:15} \t{} \t{:3}".format('*'*self.grade,
                                                                 self.name,
                                                                 self.phone,
                                                                 self.email,
                                                                 self.age,
                                                                 self.etc)
def to_member(*row): #튜플 데이러를 매개변수로 받아 Member형 객체로 return 
    return Member(row[0], row[1], row[2], row[3], row[4], row[5])

In [3]:
def insert_member():
    '사용자로부터 이름, 전화, 이메일, 나이, 등급(1~5), 기타특징 입력받아 DB에 추가한다'
    cursor = conn.cursor()
    name = input('이름 : ')
    phone = input("전화번호 : ")
    email = input("이메일 : ")
    try:
        age = int(input("나이 : "))
        if age<0:
            age = 0
    except ValueError as e:
        print('유효하지 않은 나이  입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade < 1 :
            grade = 1
        if grade > 5 :
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("기타 정보 : ")
    member = Member(name, phone, email, age, grade, etc)
    cursor.execute('''
        INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)
    ''', member.as_dict())
    conn.commit()
    print(member, "님 데이터 저장완료")
    cursor.close()   

In [4]:
def select_all():
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{:^5}\t{:^3}\t{:^10}{:^15}\t{:^3}\t{}".format("GRADE",
                    "이름", "전화", "메일", "나이", "기타"))
    print('='*70)
    for member in members:
        print(to_member(*member))
    if len(members)==0:
        print("저장된 회원이 없습니다") 
    print('='*70)
    cursor.close()

In [5]:
def select_name():
    cursor = conn.cursor()
    name = input("조회할 이름 입력 : ")
    sql = "SELECT * FROM MEMBER WHERE NAME=UPPER(:name)"    
    cursor.execute(sql, {'name':name})
    member_list = cursor.fetchall()        
    if len(member_list): 
        for row in member_list:
            member = to_member(*row)
            print(member)  
    else:
        print("조회된 결과가 없습니다.")
    cursor.close()

In [6]:
def delete_email():
    cursor = conn.cursor()
    email = input('삭제하실 이메일 : ')
    cursor.execute('DELETE FROM MEMBER WHERE EMAIL=:email',{'email':email})
    conn.commit()
    if cursor.rowcount > 0:
        print("{} 해당 이메일을 가진 고객을 삭제 했습니다.".format(email))
    else:
        print("{} 해당 이메일을 가진 고객을 찾을수 없습니다".format(email))
    cursor.close()

In [7]:
def save_csv():
    'sql 실행 결과를 튜플 리스트로 받아와서 db결과 title(description)과 튜플을 csv에 출력'
    import csv
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    fieldnames = [description[0] for description in cursor.description ]
    cursor.close()
    file_name = input('파일명을 입력하세요(csv 확장자 포함)')
    with open('data/'+file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(members)

In [8]:
def close_sql():
    global conn
    conn.close()
    print("종료합니다")

In [ ]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",
             sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/ch10_data.db')
    main()            

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
***** 곽지성 010-1234-5678   k@j.com         	18 	18 
***** 김지환 010-4571-1234   q@q.com         	18 	18 
***** 홍길동 010-1895-1823   i@i.com         	18 	18 
***** 에베베 010-4562-5852   q@p.com         	18 	18 
   ** 지대한 010-4518-1822   qe@na.com       	80 	afsjkle
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.
                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
***** 곽지성 010-1234-5678   k@j.com         	18 	18 
***** 김지환 010-4571-1234   q@q.com         	18 	18 
***** 홍길동 010-1895-1823   i@i.com         	18 	18 
***** 에베베 010-4562-5852   q@p.com         	18 	18 
   ** 지대한 010-451

## 2

In [1]:
import cx_Oracle
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")

In [16]:
cursor = conn.cursor()
sql = "CREATE TABLE MEMBER( \
            NAME VARCHAR2(30), \
            PHONE VARCHAR2(30), \
            EMAIL VARCHAR2(30), \
            AGE NUMBER(2), \
            GRADE NUMBER(2), \
            ETC VARCHAR2(30) \
)"


In [2]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def as_dict(self): #insert할 때 사용
        return {"name":self.name, "phone":self.phone, 
                "email":self.email, "age":self.age, "grade":self.grade, 
                "etc":self.etc}
    def __str__(self):
        return "{:>5} {:3} {:15} {:15} \t{} \t{:3}".format('*'*self.grade,
                                                                 self.name,
                                                                 self.phone,
                                                                 self.email,
                                                                 self.age,
                                                                 self.etc)
def to_member(*row): #튜플 데이러를 매개변수로 받아 Member형 객체로 return 
    return Member(row[0], row[1], row[2], row[3], row[4], row[5])

In [3]:
def insert_member():
    '사용자로부터 이름, 전화, 이메일, 나이, 등급(1~5), 기타특징 입력받아 DB에 추가한다'
    cursor = conn.cursor()
    name = input('이름 : ')
    phone = input("전화번호 : ")
    email = input("이메일 : ")
    try:
        age = int(input("나이 : "))
        if age<0:
            age = 0
    except ValueError as e:
        print('유효하지 않은 나이  입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade < 1 :
            grade = 1
        if grade > 5 :
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("기타 정보 : ")
    sql = """
        INSERT INTO MEMBER (NAME, PHONE, EMAIL, AGE, GRADE, ETC) 
        VALUES (:name, :phone, :email, :age, :grade, :etc)
    """
    cursor.execute(sql, {'name': name, 'phone': phone, 'email': email, 'age': age, 'grade': grade, 'etc': etc})
    conn.commit()
    print(f"{name} 님 데이터 저장 완료")
    cursor.close() 

In [4]:
def select_all():
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER"
    cursor.execute(sql)
    members = cursor.fetchall()
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{:^5}\t{:^3}\t{:^10}{:^15}\t{:^3}\t{}".format("GRADE",
                    "이름", "전화", "메일", "나이", "기타"))
    if len(members) == 0:
        print("저장된 회원이 없습니다")
    else:
        for member in members:
            print(to_member(*member))
    print('='*70)
    cursor.close()

In [5]:
def select_name():
    cursor = conn.cursor()
    name = input("조회할 이름 입력 : ")
    sql = "SELECT * FROM MEMBER WHERE NAME=UPPER(:name)"    
    cursor.execute(sql, {'name':name})
    members = cursor.fetchall()       
    if len(members): 
        for row in members:
            member = to_member(*row)
            print(member)  
    else:
        print("조회된 결과가 없습니다.")
    cursor.close()

In [6]:
def delete_email():
    cursor = conn.cursor()
    email = input('삭제하실 이메일 : ')
    sql = 'DELETE FROM MEMBER WHERE EMAIL=:email'
    cursor.execute(sql, {'email':email})
    members = cursor.fetchall()
    conn.commit()
    if cursor.rowcount > 0:
        print("{} 해당 이메일을 가진 고객을 삭제 했습니다.".format(email))
    else:
        print("{} 해당 이메일을 가진 고객을 찾을수 없습니다".format(email))
    cursor.close()

In [7]:
def save_csv():
    'sql 실행 결과를 튜플 리스트로 받아와서 db결과 title(description)과 튜플을 csv에 출력'
    import csv
    cursor = conn.cursor()
    sql = 'SELECT * FROM MEMBER'
    cursor.execute(sql)
    fieldnames = [description[0] for description in cursor.description ]
    cursor.close()
    file_name = input('파일명을 입력하세요(csv 확장자 포함)')
    with open('data/'+file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)

In [8]:
def close_sql():
    global conn
    conn.close()
    print("종료합니다")

In [9]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",
             sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/ch10_data.db')
    main()            

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
***** 곽지성 010-1234-5678   k@j.com         	18 	18 
***** 김지환 010-4571-1234   q@q.com         	18 	18 
***** 홍길동 010-1895-1823   i@i.com         	18 	18 
***** 에베베 010-4562-5852   q@p.com         	18 	18 
   ** 지대한 010-4518-1822   qe@na.com       	80 	afsjkle
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 1
이름 : 김형진
전화번호 : 010-1818-1818
이메일 : 18@18.com
나이 : 18
고객등급(1~5) : 1
기타 정보 : 18
김형진 님 데이터 저장 완료
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
조회할 이름 입력 : 김형진
    * 김형진 010-1818-1818   18@18.com       	18 	18 
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 4
삭제하실 이메일 : 18@18.com
18@18.com 해당 이메일을 가진 고객을 삭제 했습니다.
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 5
파일명을 입력하세요(csv 확장자 